#__Mount storage drive with Rclone__

In [ ]:
#@markdown <h3>←📝 Upload an rclone config <i>(rclone.conf)</i></h3>
#@markdown <h4><i>Note: Run this before using Rclone</i></4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>←📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "YourDrive" #@param {type:"string"}
local_mount_location = "/content/udrive1/" #@param {type:"string"}
# ============================= FORM ============================= #

import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M & 
  


# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

#__Mount with Google Drive File Stream__

In [ ]:
#@markdown <h3>Mount GDrive to /content/drive</h3><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# **<font color='blue'>__Google Drive to MEGA file transfer__</font>**
# <img src='https://i.ibb.co/drrqpm2/pcgamescollections2021-google-drive-to-mega.png' height="60" alt="MEGA-logo"/>

In [ ]:
# @markdown <h3>Input your Mega Id and Password</h3><br>


from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = "your email.com"  # @param {type:"string"}
PASSWORD = "your password"  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")


In [ ]:

# @markdown <h3>←TRANSFER NOW</h3><br>
# TRANSFER NOW

import time
import subprocess
import contextlib
from IPython.display import clear_output

PATH_TO_FILE = "/content/drive/"  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")


In [ ]:
# @markdown <h3>←LOGOUT</h3><br>

!mega-logout

# **<font color='blue'>__ARCHIVE.ORG UPLOADER__</font>**

In [ ]:
#@markdown <h3>←INSTALL INTERNETARCHIVE</h3>
!pip install internetarchive
!curl -LO https://archive.org/download/ia-pex/ia
!chmod +x ia

In [ ]:
#@markdown <h3>←INPUT CREDENTIALS</h3>
!ia configure

In [ ]:
#@markdown <h3>←START UPLOAD</h3>
File_path = "" #@param {type:"string"}
Identifier = "ABCDEXZYWV" #@param {type:"string"}
media_type = "movies" #@param ["movies", "documents"]
!ia upload {Identifier} {File_path} --metadata="mediatype:{media_type}" --retries 10

# **<font color='blue'>__MEDIAFIRE UPLOADER__</font>**

In [ ]:
#@markdown <h3>👈 Run this cell to install requirements</h3>
!echo -n "Installing mediafire-ul..." && mkdir -p sample_data/mediafire-ul &> /dev/null && git clone https://github.com/JayBrown/mfcmd.py---MediaFire-uploader.git sample_data/mediafire-ul &> /dev/null && pip install pymediafire &> /dev/null && pip install mediafire &> /dev/null && echo " Done!"


In [ ]:
#@markdown <h3>⬆️ MediaFire Uploader</h3>
#@markdown The MediaFire API is pretty buggy, and you might receive an error, because the server hasn't finished calculating the file's checksum yet; in that case, wait a couple of seconds (depending on the filesize) and repeat the command: it will not upload, but instead detect the already uploaded file and return the download URL.
Email_Address = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
File_path = "" #@param {type:"string"}
mfemail, mfpassword = Email_Address, Password

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

filename, pathname = get_file_info(File_path)
print("Uploading", filename+"...")
!cd /content/sample_data/mediafire-ul && python3 mfcmd.py -e $mfemail -p $mfpassword -f $File_path 2>/dev/null
print("Done!")

# <img src="https://img.icons8.com/dotty/80/000000/archive.png" height="45" align="left" alt=""/><img src="" height="45" align="left" alt=""/><b><font size=6>Create / Extract Archive</font></b>

##__Rar Files/Folders__

In [ ]:
# @markdown <h3>←INSTALL RAR</h3><br>
!apt-get install rar

In [ ]:
# @markdown <h3>←CREATE ARCHIVE</h3><br>
Input_File_or_Folder_Path = ""  # @param {type:"string"}
Output_File_Name = ""  # @param {type:"string"}
Split_Volume = "0"  # @param {type:"string"}

!rar a -v{Split_Volume}m "{Input_File_or_Folder_Path}/{Output_File_Name}" "{Input_File_or_Folder_Path}"

#@markdown #####_*Split_Volume Sample:_
#@markdown #####_100 = 100 MegaBytes._